# Allocate emissions for IHS processes and calculate implied conversion factors

In [1]:
# Import packages
import itertools
import numpy as np
import pandas as pd

pd.set_option('max_columns', None)
pd.options.mode.chained_assignment = None

## Data imports

In [2]:
input_path = '../data/'

material_emissions_path = input_path+'combined/ihsMaterialsEmissions_w_uncertainties.csv'
ei_emissions_path = input_path+'processed/conversionFactors_ecoinvent_grouped.csv'
mass_to_energy_path = input_path+'extra_inputs/chemical energy value.xlsx'
mass_to_price_path = input_path+'extra_inputs/PEP Yearbook Price History.xlsm'

output_path = '../data/combined/'

In [3]:
material_emissions = pd.read_csv(material_emissions_path, index_col=0)

prods = pd.read_csv('C:/Users\lukec\PycharmProjects\petrochemical-data\data\extra_inputs\product_groups.csv', index_col=0)
intermeds = prods[[i in ['Organic chemicals', 'Primary chemicals', 'Other intermediates'] for i in prods['Product group']]]['PRODUCT']
equivs_dict = dict(zip(prods['PRODUCT'], prods['Product group']))

# Ensure names are related for intermediates
name_matching_ints = {'PROPYLENE, CHEM GRADE':'PROPYLENE', 'PROPYLENE, REFINERY GRADE':'PROPYLENE', 'PROPYLENE, POLYMER GRADE':'PROPYLENE',
 'P-XYLENE':'PARA-XYLENE', 'O-XYLENE':'ORTHO-XYLENE', 'XYLENES, MIXED':'MIXED XYLENES','METHANOL, CRUDE':'METHANOL'}
material_emissions['Source/Object'] = material_emissions['Source/Object'].replace(name_matching_ints)

# Change type tpo match intermediates as types
material_emissions.loc[(material_emissions['Type'] == 'Raw Material') & (material_emissions['Source/Object'].isin(intermeds)), 'Type'] = 'Diff'
material_emissions.loc[(material_emissions['Type'] == 'Diff'), 'Type'] = material_emissions.loc[(material_emissions['Type'] == 'Diff')]['Source/Object'].replace(equivs_dict)

material_emissions.head()

,Code,Data Version,Source/Object,Type,Target/Process,Research Year,Country/Reg,Product,Value,Value unit,Capacity unit,MeasType,Provenance,Value_sigma,ei_match,ei_CO2e_20a_cradle-to-gate,ei_CO2e_20a_cradle-to-gate_sigma,ei_CO2e_20a_conv_factor,ei_CO2e_20a_conv_factor_sigma,ei_CO2e_100a_cradle-to-gate,ei_CO2e_100a_cradle-to-gate_sigma,ei_CO2e_100a_conv_factor,ei_CO2e_100a_conv_factor_sigma,ei_Carbon dioxide_cradle-to-gate,ei_Carbon dioxide_cradle-to-gate_sigma,ei_Carbon dioxide_conv_factor,ei_Carbon dioxide_conv_factor_sigma,ei_Carbon monoxide_cradle-to-gate,ei_Carbon monoxide_cradle-to-gate_sigma,ei_Carbon monoxide_conv_factor,ei_Carbon monoxide_conv_factor_sigma,ei_Chloroform_cradle-to-gate,ei_Chloroform_cradle-to-gate_sigma,ei_Chloroform_conv_factor,ei_Chloroform_conv_factor_sigma,ei_Dinitrogen monoxide_cradle-to-gate,ei_Dinitrogen monoxide_cradle-to-gate_sigma,ei_Dinitrogen monoxide_conv_factor,ei_Dinitrogen monoxide_conv_factor_sigma,ei_Ethane_cradle-to-gate,ei_Ethane_cradle-to-gate_sigma,ei_Ethane_conv_factor,ei_Ethane_conv_factor_sigma,ei_Methane_cradle-to-gate,ei_Methane_cradle-to-gate_sigma,ei_Methane_conv_factor,ei_Methane_conv_factor_sigma,ei_Nitric oxide_cradle-to-gate,ei_Nitric oxide_cradle-to-gate_sigma,ei_Nitric oxide_conv_factor,ei_Nitric oxide_conv_factor_sigma,ei_Nitrogen fluoride_cradle-to-gate,ei_Nitrogen fluoride_cradle-to-gate_sigma,ei_Nitrogen fluoride_conv_factor,ei_Nitrogen fluoride_conv_factor_sigma,ei_Perfluoropentane_cradle-to-gate,ei_Perfluoropentane_cradle-to-gate_sigma,ei_Perfluoropentane_conv_factor,ei_Perfluoropentane_conv_factor_sigma,ei_Sulfur hexafluoride_cradle-to-gate,ei_Sulfur hexafluoride_cradle-to-gate_sigma,ei_Sulfur hexafluoride_conv_factor,ei_Sulfur hexafluoride_conv_factor_sigma,ei_Other_cradle-to-gate,ei_Other_cradle-to-gate_sigma,ei_Other_conv_factor,ei_Other_conv_factor_sigma,cm_match,cm_CO2e_20a_cradle-to-gate,cm_CO2e_20a_cradle-to-gate_sigma,cm_CO2e_20a_conv_factor,cm_CO2e_20a_conv_factor_sigma,cm_CO2e_100a_cradle-to-gate,cm_CO2e_100a_cradle-to-gate_sigma,cm_CO2e_100a_conv_factor,cm_CO2e_100a_conv_factor_sigma,cm_Carbon dioxide_cradle-to-gate,cm_Carbon dioxide_cradle-to-gate_sigma,cm_Carbon dioxide_conv_factor,cm_Carbon dioxide_conv_factor_sigma,cm_Carbon monoxide_cradle-to-gate,cm_Carbon monoxide_cradle-to-gate_sigma,cm_Carbon monoxide_conv_factor,cm_Carbon monoxide_conv_factor_sigma,cm_Chloroform_cradle-to-gate,cm_Chloroform_cradle-to-gate_sigma,cm_Chloroform_conv_factor,cm_Chloroform_conv_factor_sigma,cm_Dinitrogen monoxide_cradle-to-gate,cm_Dinitrogen monoxide_cradle-to-gate_sigma,cm_Dinitrogen monoxide_conv_factor,cm_Dinitrogen monoxide_conv_factor_sigma,cm_Ethane_cradle-to-gate,cm_Ethane_cradle-to-gate_sigma,cm_Ethane_conv_factor,cm_Ethane_conv_factor_sigma,cm_Methane_cradle-to-gate,cm_Methane_cradle-to-gate_sigma,cm_Methane_conv_factor,cm_Methane_conv_factor_sigma,cm_Nitric oxide_cradle-to-gate,cm_Nitric oxide_cradle-to-gate_sigma,cm_Nitric oxide_conv_factor,cm_Nitric oxide_conv_factor_sigma,cm_Nitrogen fluoride_cradle-to-gate,cm_Nitrogen fluoride_cradle-to-gate_sigma,cm_Nitrogen fluoride_conv_factor,cm_Nitrogen fluoride_conv_factor_sigma,cm_Perfluoropentane_cradle-to-gate,cm_Perfluoropentane_cradle-to-gate_sigma,cm_Perfluoropentane_conv_factor,cm_Perfluoropentane_conv_factor_sigma,cm_Sulfur hexafluoride_cradle-to-gate,cm_Sulfur hexafluoride_cradle-to-gate_sigma,cm_Sulfur hexafluoride_conv_factor,cm_Sulfur hexafluoride_conv_factor_sigma,cm_Other_cradle-to-gate,cm_Other_cradle-to-gate_sigma,cm_Other_conv_factor,cm_Other_conv_factor_sigma
3801,2M-195,2021 Q3,COOLING WATER,Direct Utilities,"1,12-DODECANEDIAMINE (DMDA) FROM 1,12-DODECANE...",1990.0,Germany,"1,12-DODECANEDIAMINE",178.474735,kg/kg,MM KG/yr,Utilities,IHS PEP,8.923737,COOLING WATER,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [4]:
## Functions to allocate emissions from materials to products output from IHS recipes

def uncertainty_propagation(calc:str, x:float, dx:float, y:float=1, dy:float=0, z:float=1, propagation_type:str='simple') -> float:
    """This function calculates uncertainty dz given x, y, dx, dy and z"""
    if calc == 'mult':
        xdiv = np.divide(dx, x, out=np.zeros_like(dx), where=x!=0)
        ydiv = np.divide(dy, y, out=np.zeros_like(dy), where=y!=0)
        if propagation_type == 'simple':
            return (xdiv + ydiv)*z
        elif propagation_type == 'stdev':
            return np.sqrt(pow(xdiv,2) + pow(ydiv,2))*z
        else: Exception('Specified propagation_type not recognised.')

    elif calc == 'add':
        if propagation_type == 'simple':
            return abs(dx)+abs(dy)
        elif propagation_type == 'stdev':
            return np.sqrt(pow(dx,2) + pow(dy,2))
        else: Exception('Specified propagation_type not recognised.')
    else: Exception('Please specify calc of propagation')

In [5]:
def filter_df_for_type(df, types, type_col):
    return df[[i in types for i in df[type_col]]]

def calculate_type_emissions(materials_df, product_df, emission_types:list, group_name:str, emissions_cols:list, emissions_cols_sigma:list, emission_val_cols:list, product_ratio_col, product_value_col, emission_type_col = 'Type'):

    # Impose lists
    if type(emission_type_col) is str:
        emission_type_col = [emission_type_col]
        emission_types = [emission_types]
    if type(emission_types) is str:
        emission_types = [emission_types]

    # Sum for groups
    grouped_df, grouped_df_sigma = materials_df.copy(), materials_df.copy()
    for emission_type_list, col in zip(emission_types, emission_type_col):
        if grouped_df.empty is False:
            grouped_df = filter_df_for_type(grouped_df, emission_type_list, col)

    group_ems = np.sum(grouped_df[emissions_cols])
    group_ems_sigma = np.sum(grouped_df[emissions_cols_sigma])

    # Loop through each value/gas column
    for val_column, gas_column, val_column_sigma, gas_column_sigma in zip(emission_val_cols, emissions_cols, [col + '_sigma' for col in emission_val_cols], emissions_cols_sigma):

        # Allocate emissions for value and uncertainty
        product_df[group_name+' allocated ' + val_column] = group_ems[gas_column]*product_df[product_ratio_col]
        product_df[group_name+' allocated ' + val_column_sigma] = uncertainty_propagation('mult', group_ems[gas_column], group_ems_sigma[gas_column_sigma], product_df[product_ratio_col], product_df[product_ratio_col+'_sigma'], z=product_df[group_name+' allocated ' + val_column])

        # Calculate emissions intensity for values and uncertainty
        product_df[group_name+' unit emission intensity ' + val_column] = product_df[group_name+' allocated ' + val_column]/product_df[product_value_col]
        product_df[group_name+' unit emission intensity ' + val_column_sigma] = uncertainty_propagation('mult', product_df[group_name+' allocated ' + val_column], product_df[group_name+' allocated ' + val_column_sigma], product_df[product_value_col], product_df[product_value_col+'_sigma'], z=product_df[group_name+' unit emission intensity ' + val_column])

    return product_df

In [6]:
def allocate_emissions(df:pd.DataFrame, emission_val_cols:list, mass_to_other_convs=False, mass_to_other_uncertainty=0.01, value_col= 'Mass, kg', ratio_col = 'Mass ratio', unit='kg'):
    # Get inputs to products
    df_ins = df[df['Type']!='By-Product']
    for column in emission_val_cols:
        df_ins['combined_' + column] = np.nanmean([df_ins['ei_' + column + '_cradle-to-gate'], df_ins['cm_' + column + '_cradle-to-gate']], axis=0)
        df_ins['combined_' + column + '_sigma'] = np.nanmean([df_ins['ei_' + column + '_cradle-to-gate_sigma'], df_ins['cm_' + column + '_cradle-to-gate_sigma']], axis=0)
    combined_cols = ['combined_' + column for column in emission_val_cols]
    combined_cols_sigma = ['combined_' + column + '_sigma' for column in emission_val_cols]

    allocation = pd.DataFrame()

    # Loop through each process
    for code in df['Code'].unique():

        # Get by-products and mass ratios
        temp = df[df['Code']==code][['Code', 'Data Version', 'Source/Object', 'Type', 'Target/Process', 'Research Year', 'Country/Reg', 'Product', 'Value', 'Value unit', 'Value_sigma', 'Capacity unit', 'MeasType', 'Provenance']]
        a = temp.iloc[0]
        a['Source/Object'], a['Type'], a['Value'], a['Value unit'], a['Value_sigma'] = a['Product'], 'Product', float(1), 'kg/kg', float(0)
        a = pd.DataFrame(a.values.reshape(1,-1), columns=a.index)
        temp = temp[temp['Type']=='By-Product']
        temp = pd.concat([temp, a], axis=0)

        # Convert values to energy if conversion exists in mass_to_enrgy_convs file
        if mass_to_other_convs is not False:
            # value_col, ratio_col, unit = 'Energy, MJ', 'Energy ratio', 'MJ'
            temp = temp.merge(mass_to_other_convs, how='left', left_on='Source/Object', right_on=mass_to_other_convs['Product'].str.upper())
            if temp['Conversion'].isnull().values.any():
                continue
            else:
                temp[value_col] = temp['Conversion']*abs(temp['Value'])
                temp[value_col+'_sigma'] = uncertainty_propagation('mult', abs(temp['Value']), temp['Value_sigma'], temp['Conversion'], mass_to_other_uncertainty*temp['Conversion'], z=temp[value_col])

        else:
            #value_col, ratio_col, unit = 'Mass, kg', 'Mass ratio', 'kg'
            temp[value_col] = abs(temp['Value'])
            temp[value_col+'_sigma'] = temp['Value_sigma']

        # Get ratio of product vs all products+by-products
        temp[ratio_col] = temp[value_col]/sum(temp[value_col])
        temp[ratio_col+'_sigma'] = uncertainty_propagation('mult', temp[value_col], temp[value_col+'_sigma'], sum(temp[value_col]), sum(temp[value_col+'_sigma']), z=temp[ratio_col])
        ### -> Assumption of adding uncertainties together for sum(temp[value_col+'_sigma'])

        # Get process emissions & allocate
        used_mats = df_ins[df_ins['Code']==code]

        unique_types = list(used_mats['Type'].unique())
        group_names = ['Total'] + unique_types
        types_lists = [unique_types]+unique_types

        for group_name, types in zip(group_names, types_lists):
            temp = calculate_type_emissions(used_mats, temp, types, group_name, combined_cols, combined_cols_sigma, emission_val_cols, ratio_col, value_col)

        temp = calculate_type_emissions(used_mats, temp, [['Indirect Utilities'], ['ELECTRICITY']], 'Electricity', combined_cols, combined_cols_sigma, emission_val_cols, ratio_col, value_col, emission_type_col = ['Type', 'Source/Object'])

        # Identify missing material emissions
        temp['Missing raw materials (>1% mass)'] = str(used_mats[(used_mats['Type']=='Raw Material') & (str(used_mats[combined_cols[0]])=='nan') & (used_mats['Value']> 0.01*np.sum(used_mats['Value']))]['Source/Object'].tolist())

        # Identify missing utility emissions
        temp['Missing utilities'] = str(used_mats[(used_mats['Type']=='Utilities') & (str(used_mats[combined_cols[0]])=='nan')]['Source/Object'].tolist())

        # Add current product to allocation list
        allocation = pd.concat([allocation, temp], axis=0)
    #return used_mats
    return allocation

In [7]:
keep_all = True

ei_emissions = pd.read_csv(ei_emissions_path, index_col=0)
if keep_all:
    emission_val_cols = list(ei_emissions.columns[3:16])
    emission_val_cols_sigma = list(ei_emissions.columns[16:])
else:
    emission_val_cols = list(ei_emissions.columns[3:5])
    emission_val_cols_sigma = list(ei_emissions.columns[16:18])


# Mass allocation
mass_allocation = allocate_emissions(material_emissions, emission_val_cols)
mass_allocation.to_csv(output_path+'ihsProcessEmissions_allgases_mass_dedoubled.csv')
#mass_allocation = pd.read_csv(output_path+'ihsProcessEmissions_w_elec_allgases.csv', index_col=0)

# Energy allocation
mass_to_energy_convs = pd.read_excel(mass_to_energy_path, skiprows=4, usecols=range(4))[['Chemical', 'HHV (GJ/t)']].rename(columns={'Chemical': 'Product', 'HHV (GJ/t)':'Conversion'})
energy_allocation = allocate_emissions(material_emissions, emission_val_cols, mass_to_energy_convs, value_col= 'Energy, MJ', ratio_col='Energy ratio', unit='MJ')
energy_allocation.to_csv(output_path+'ihsProcessEmissions_allgases_energy_dedoubled.csv')
#energy_allocation.read_csv(output_path+'ihsProcessEmissions_allgases_energy.csv', index_col=0)

# Economic allocation
mass_to_price = pd.read_excel(mass_to_price_path, sheet_name='Germany metric')[1:].rename(columns={'Unnamed: 0': 'Product', 'Q4 2021':'Conversion'})[['Product', 'Conversion']]
economic_allocation = allocate_emissions(material_emissions, emission_val_cols, mass_to_price, value_col= 'Cost, c', ratio_col='Cost ratio', unit='c', mass_to_other_uncertainty=0.1)
economic_allocation.to_csv(output_path+'ihsProcessEmissions_allgases_economic_dedoubled.csv')

# mass_allocation = pd.read_csv(output_path+'ihsProcessEmissions_allgases_mass.csv', index_col=0)
# energy_allocation = pd.read_csv(output_path+'ihsProcessEmissions_allgases_energy.csv', index_col=0)
# economic_allocation = pd.read_csv(output_path+'ihsProcessEmissions_allgases_economic.csv', index_col=0)

# Merge allocations into dataframe
energy_allocation.rename(columns={'Product_x':'Product'}, inplace=True)
economic_allocation.rename(columns={'Product_x':'Product'}, inplace=True)
mass_and_energy_allocation = mass_allocation.merge(energy_allocation, on=list(mass_allocation.columns[:13].values), how='left', suffixes=('_mass', '_energy'))
mass_and_energy_allocation.columns = [i+'_mass' if 'Other intermediates' in i else i for i in mass_and_energy_allocation.columns]

economic_allocation.columns = list(economic_allocation.columns[:20])+[i+'_economic' for i in economic_allocation.columns[20:]]
total_allocation = mass_and_energy_allocation.merge(economic_allocation, on=list(mass_allocation.columns[:13].values), how='left')

C:\Users\lukec\AppData\Local\Temp/ipykernel_24532/2636808998.py:5: RuntimeWarning: Mean of empty slice
  df_ins['combined_' + column] = np.nanmean([df_ins['ei_' + column + '_cradle-to-gate'], df_ins['cm_' + column + '_cradle-to-gate']], axis=0)
C:\Users\lukec\AppData\Local\Temp/ipykernel_24532/2636808998.py:6: RuntimeWarning: Mean of empty slice
  df_ins['combined_' + column + '_sigma'] = np.nanmean([df_ins['ei_' + column + '_cradle-to-gate_sigma'], df_ins['cm_' + column + '_cradle-to-gate_sigma']], axis=0)
C:\Users\lukec\AppData\Local\Temp/ipykernel_24532/2990847033.py:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  product_df[group_name+' allocated ' + val_column_sigma] = uncertainty_propagation('mult', group_ems[gas_column], group_ems_sigma[gas_column

In [8]:
total_allocation.head()

Code Data Version            Source/Object        Type  \
0   2M-195      2021 Q3     1,12-DODECANEDIAMINE     Product   
1   2M-197      2021 Q3  1,12-DODECANEDIOIC ACID     Product   
2   2M-196      2021 Q3  1,12-DODECANEDIOIC ACID     Product   
3  2M-1611      2021 Q3           C4 RAFFINATE 2  By-Product   
4  2M-1611      2021 Q3               HEAVY ENDS  By-Product   

                                      Target/Process Research Year  \
0  1,12-DODECANEDIAMINE (DMDA) FROM 1,12-DODECANE...        1990.0   
1             1,12-DODECANEDIOIC ACID FROM BUTADIENE        1990.0   
2  1,12-DODECANEDIOIC ACID FROM CYCLODODECATRIENE...        1971.0   
3  1,3 BUTADIENE BY OXIDATIVE DEHYDROGENATION OF ...        2014.0   
4  1,3 BUTADIENE BY OXIDATIVE DEHYDROGENATION OF ...        2014.0   

  Country/Reg                  Product     Value Value unit Value_sigma  \
0     Germany     1,12-DODECANEDIAMINE       1.0      kg/kg         0.0   
1     Germany  1,12-DODECANEDIOIC ACID       1.0      kg/kg         0.0   
2     Germany  1,12-DODECANEDIOIC ACID       1.0      kg/kg         0.0   
3     Germany            1,3-BUTADIENE  -1.04354      kg/kg    0.052177   
4     Germany            1,3-BUTADIENE -0.007466      kg/kg    0.000373   

  Capacity unit   MeasType Provenance_mass  Mass, kg Mass, kg_sigma  \
0      MM KG/yr  Utilities         IHS PEP       1.0            0.0   
1      MM KG/yr  Utilities         IHS PEP       1.0            0.0   
2      MM KG/yr  Utilities         IHS PEP       1.0            0.0   
3      MM KG/yr     Recipe         IHS PEP   1.04354       0.052177   
4      MM KG/yr     Recipe         IHS PEP  0.007466       0.000373   

  Mass ratio Mass ratio_sigma Total allocated CO2e_20a_mass  \
0        1.0              0.0                      2.593065   
1        1.0              0.0                       2.78796   
2        1.0              0.0                      6.967293   
3   0.246508         0.021739                      0.782174   
4   0.001764         0.000156                      0.005596   

  Total allocated CO2e_20a_sigma_mass  \
0                            0.317109   
1                            0.410284   
2                            1.045094   
3                            0.184868   
4                            0.001323   

  Total unit emission intensity CO2e_20a_mass  \
0                                    2.593065   
1                                     2.78796   
2                                    6.967293   
3                                    0.749539   
4                                    0.749539   

  Total unit emission intensity CO2e_20a_sigma_mass  \
0                                          0.317109   
1                                          0.410284   
2                                          1.045094   
3                                          0.214632   
4                                          0.214632   

  Total allocated CO2e_100a_mass Total allocated CO2e_100a_sigma_mass  \
0                       2.350622                             0.280742   
1                       2.377028                             0.348644   
2                       6.347768                             0.952165   
3                       0.711411                             0.168013   
4                        0.00509                             0.001202   

  Total unit emission intensity CO2e_100a_mass  \
0                                     2.350622   
1                                     2.377028   
2                                     6.347768   
3                                     0.681728   
4                                     0.681728   

  Total unit emission intensity CO2e_100a_sigma_mass  \
0                                           0.280742   
1                                           0.348644   
2                                           0.952165   
3                                            0.19509   
4                                            0.195

In [9]:
# Output to file
total_allocation.to_csv(output_path+'ihsProcessEmissions_allgases_allallocs_dedoubled.csv')
#total_allocation.to_csv(output_path+'ihsProcessEmissions_2gases_allallocs.csv')

In [9]:
## Output most needed matches

# vals = [sublist.split('[')[1].split(']')[0].replace("'","") for sublist in list(allocation['Missing raw materials (>1% mass)'].values)]
# pd.DataFrame(np.array([vals,vals]).transpose(), columns=['Type','val']).groupby('Type').count().sort_values('val', ascending=False).to_csv('matches_needed.csv')

## Calculate implied emissions factors from allocation

In [10]:
emission_val_cols = list(ei_emissions.columns[3:16])
total_allocation = pd.read_csv(output_path+'ihsProcessEmissions_allgases_allallocs_dedoubled.csv', index_col=0)

total_allocation = total_allocation[total_allocation['Type']=='Product']

suffixes = ['_mass', '_energy', '_economic']

IHS_emissions = total_allocation[['Target/Process', 'Product']+[i for i in total_allocation.columns if 'allocated' in i]]
IHS_emissions.rename(columns={'Target/Process':'ihs_match'}, inplace=True)

group_names = ['Total', 'Raw Material', 'Organic chemicals', 'Primary chemicals', 'Other intermediates', 'Direct Utilities', 'Indirect Utilities', 'Direct Process', 'Electricity']
output_names = ['ihs_cradle-to-out-gate ', 'Feedstock ', 'Organic chemicals ', 'Primary chemicals ', 'Other intermediates ', 'Direct Utilities ', 'Indirect Utilities ', 'Direct Process ', 'Electricity ']

# group_names = ['Total', 'CtG', 'GtG', 'Raw Material', 'Direct Utilities', 'Indirect Utilities', 'Direct Process', 'Electricity']
# output_names = ['ihs_cradle-to-out-gate ', 'ihs_cradle-to-in-gate ', 'ihs_gate-to-gate ', 'Raw Material ', 'Direct Utilities ', 'Indirect Utilities ', 'Direct Process ', 'Electricity ']

for suffix in suffixes:
    for group_name, output_name in zip(group_names, output_names):
        catch = [IHS_emissions.rename(columns={group_name+' allocated '+col+suffix: output_name+col+', '+suffix[1:]+' allocation factor', group_name+' allocated '+col+'_sigma'+suffix: output_name+col+', '+suffix[1:]+' allocation sigma'}, inplace=True) for col in emission_val_cols]

mats = material_emissions.rename(columns={'Product':'Target','Source/Object':'Product'}).drop_duplicates(subset=['Product']).drop(columns=['Code', 'Data Version', 'Type', 'Target/Process', 'Research Year', 'Country/Reg', 'Target', 'Value', 'Value unit', 'Value_sigma', 'Capacity unit', 'MeasType', 'Provenance'])

combined_factors = mats.sort_values('Product').reset_index(drop=True)
combined_factors = combined_factors.merge(IHS_emissions, on='Product', how='outer')
combined_factors = combined_factors.dropna(subset=combined_factors.columns[1:], how='all').reset_index(drop=True)

C:\Users\lukec\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (489,490,808,809) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [11]:
# Integrate IFA conversion factors
ifa_factors = pd.read_csv(input_path+'extracted/conversionFactors_from_IFA.csv', index_col=0)
ifa_factors['Product'] = ifa_factors['Product'].str.upper()
ifa_matches = pd.read_csv(input_path+'extra_inputs/ifa_matches.csv')
ifa_factors = ifa_factors.merge(ifa_matches.dropna(), on='Product', how='right').drop(columns='Product').rename(columns={'Match':'Product'})

combined_factors = combined_factors.merge(ifa_factors, on='Product', how='left')

In [12]:
# Output file
combined_factors.sort_values('Product').to_csv(output_path+'processConversionFactors_allgases_allalloc_dedoubled.csv', index=False)

In [ ]:
# Output condensed file
combined_factors[['Product', 'ei_match', 'ei_CO2e_100a_conv_factor', 'cm_match', 'cm_CO2e_100a_conv_factor', 'ihs_match', 'ihs_cradle-to-out-gate CO2e_100a, mass allocation factor']]
#condensed.to_csv(output_path+'processConversionFactors_condensed.csv', index=False)